# EDA данных

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.feather as ft
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime

warnings.simplefilter('ignore')

## Загрузим данные

In [ ]:
df_test = ft.read_feather('project_data_imv_auto/project_test_public.f')
df_test.head(3)

In [ ]:
df_train = ft.read_feather('project_data_imv_auto/project_train.f')
df_train.head(3)

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
# тест и трейн по составу колонок отличаются следующим образом:
set(df_train.columns).difference(set(df_test.columns))

In [ ]:
options = pd.read_csv('project_data_imv_auto/option_names.csv')
options.head(3)

In [ ]:
df_train['price'] = df_train['price'].astype(float)

### Generation

In [ ]:
df_train['generation'].describe()

In [ ]:
df_train['generation'].unique()

In [ ]:
df_train['generation'].str.split().apply(len).value_counts()

В основном колонка заполнена данными вида "кодировка поколения" + "годы", но в трети объявлений так же содержится информация о рестайлинге и поколении рестайлинга:

In [ ]:
df_train[df_train['generation'].str.split().apply(len) == 4]['generation'].unique()

Попробуем разнести информацию по разным колонкам и посмотрим на их связь с таргетом.

In [ ]:
# Оставим в колонке generation только кодировку поколения.
# Информацию о рестайлинге вынесем в колонку restyling. При этом поколение без рестайлинга будем кодировать как 0, 
# рестайлинг первого поколения как 1, второго поколения рестайлинга как 2 и тд.
# Наконец вынесем годы в колонку generation_years
def restyling_extract(gen_list: list) -> int:
    """
    Выделяем поколение рестайлинга из списка слов колонки generation
    """
    if len(gen_list) == 4:
        return int(gen_list[-2])
    elif len(gen_list) == 3:
        return 1
    return 0

generation = df_train['generation'].str.split()
df_train['generation_old'] = df_train['generation']
df_train['generation'] = generation.apply(lambda x: x[0])
df_train['generation_years'] = generation.apply(lambda x: x[-1])
df_train['restyling'] = generation.apply(lambda x: restyling_extract(x))

In [ ]:
fig = sns.catplot(data=df_train[['restyling', 'price']], x="restyling", y="price", kind="box", height=6, aspect=2)
fig.set(ylim=(0, 1.5e7))

In [ ]:
df_train['generation'].nunique(), df_train['generation_years'].nunique()

#### summary
Выделив дополнительные данные из колонки generation нам удалось несколько уменьшить количество категорий - возможно это поможет в дальнейшем при обучении модели. При этом как видим количество уникальных значений generation и generation_years практически идентично - возможно это связано с тем, что годы выпуска каждого поколения фактически определяют и само поколение. В последующем необходимо будет проверить это.

### Modification

In [ ]:
df_train['modification'].describe()

In [ ]:
df_train['modification'].unique()

In [ ]:
# Выделим из данной колонки объем двигателя, заполним образующиеся nan'ы данными из гугла

df_train['engine_volume'] = df_train.modification.str.extract(r'(?P<engine_volume>\d\.\d)')
df_train.loc[df_train['modification'] == 'FX30d 4WD AT (238 л.с.)', 'engine_volume'] = '3.0'
df_train.loc[df_train['modification'] == 'P85', 'engine_volume'] = '0.0'

In [ ]:
df_train['engine_volume'].describe()

In [ ]:
sns.histplot(data=df_train['engine_volume'].astype(float), binwidth=0.1)

In [ ]:
# Выделим из данной колонки количество лошадиных сил, заполним образующиеся nan'ы данными из гугла

df_train['horse_power'] = df_train.modification.str.extract(r'(?P<horse_power>\(.*\))')
df_train['horse_power'] = df_train['horse_power'].str.strip('( л.с.)')
df_train['horse_power'] = df_train['horse_power'].fillna('382')
df_train['horse_power'] = df_train['horse_power'].astype(int)

In [ ]:
df_train['horse_power'].hist(bins=100)

#### summary

Выделив дополнительные колонки из modification мы получили дополнительные данные об объеме двигателя и его мощности. Остальная информация, содержащая по большей части данные о коробке передач, типе привода и тд будем считать избыточной и отбросим.

### Equipment

In [ ]:
df_train['equipment'].describe()

In [ ]:
df_train['equipment'].isna().sum()

In [ ]:
df_train['equipment'].unique()

In [ ]:
df_train['equipment'].fillna('Базовая').str.split().apply(len).value_counts()

In [ ]:
df_train[df_train['equipment'].fillna('None').str.split().apply(len) == 1]['equipment'].unique()

In [ ]:
df_train[df_train['equipment'].fillna('None').str.split().apply(len) == 3]['equipment'].unique()

In [ ]:
df_train[df_train['equipment'].fillna('None').str.split().apply(len) == 5]['equipment'].unique()

In [ ]:
df_train[df_train['equipment'].fillna('None').str.split().apply(len) == 7]['price']

Как видно, большое количество уникальных значений усложняет работу с ними, при этом часть информации, зашифрованной в строке дублирует уже существующую:
'Люкс Премиум Автомат', 'GLE 53 4MATIC+', 'Top Prestige (2015-2016)'

In [ ]:
# Посмотрим, можно ли выделить информацию о количестве мест
(df_train.equipment.str.extract(r'(?P<equipment>[0-9]?\sмест)').notna().sum(), 
df_train.equipment.str.extract(r'(?P<equipment>[0-9]?\sseats)').notna().sum())

In [ ]:
# Посмотрим на количество записей с упоминанием о локальной сборке

df_train.equipment.str.extract(r'(?P<equipment>Локальная сборка)').notna().sum()

#### summary

Очень большое количество категорий. 
Возможные пути улучшения качества данной колонки:
* Убрать опечатки: есть категории "Luxary" и "Confort", "Standart" т.д.
* Убрать синонимы, как внутри англоязычных и русскоязычных строк: "Basic" и "Base", "Lux" и "Luxe", так и между разными языками - "Lux" и "Люкс", "Premium" и "Премиум" и тд
* Обрабатывать данную колонку как мультилейбл - возможно преобразование типа mean-target encoding с учетом мультилейблов даст модели лучшее представление о влиянии этой колонки на таргет:
"Family + Style + High-Tech" можно представить как три лейбла - "Family", "Style" и "High-Tech"

### Description

In [ ]:
df_train['description'].describe()

In [ ]:
df_train['description'].value_counts()[-1000:-200]

#### summary

Очень большое количество категорий, много вариантов обработки. 
Возможные пути обработки текстов описаний:
* Убрать самые частые как неинформативные - "Торг", "Все вопросы по телефону", "." и т.д.
* Судя по описаниям - объявления с очень подробным и одинаковым описанием написаны от дилерских центров или перекупов. Возможно имеет смысл выделить в отдельную колонку информацию об этом.
* Получение эмбеддингов описаний: tf-idf, word2vec c, например, последующией кластеризацией.

### Audiosystem

In [ ]:
df_train['audiosistema'].describe()

In [ ]:
mask = {str(i): options[options['id'] == i].viewItemLabel.values[0] for i in df_train['audiosistema'].dropna().unique()}
mask['nan'] = 'Нет аудиосистемы'

In [ ]:
mask

In [ ]:
df_train['audiosistema'] = df_train['audiosistema'].apply(lambda x: mask[str(x)])

In [ ]:
df_train['audiosistema'].value_counts()

In [ ]:
fig = sns.catplot(data=df_train[['audiosistema', 'price']], x="audiosistema", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 0.6e7))

#### summary

Категория довольно сильно связана с таргетом - распределения довольно сильно отличаются в зависимости от наличия и типа аудиосистемы.

### Wheels
####  discs

In [ ]:
df_train['diski'].describe()

In [ ]:
mask = {str(i): int(options[options['id'] == i].viewItemLabel.values[0][:-1]) for i in df_train['diski'].dropna().unique()}
mask['nan'] = -1

In [ ]:
mask

In [ ]:
df_train['diski'] = df_train['diski'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['diski', 'price']], x="diski", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 3e7))
plt.show()

In [ ]:
df_train['diski'].value_counts()

In [ ]:
# Построим гистрограмму без неизвестных значений
fig = sns.histplot(data=df_train[df_train['diski'] != -1]['diski'], discrete=True)

#### Summary

Больше половины всех значений - пропуски, при этом там, где мы знаем о величине дисков видно, что есть взаимосвязь с таргетом.
Возможно будет иметь смысл размечать пропуски, например модой по данной модели автомобиля.

### Power windows
#### electropodemniki

In [ ]:
df_train['electropodemniki'].describe()

In [ ]:
df_train['electropodemniki'].value_counts()

In [ ]:
mask = {str(i): options[options['id'] == i].viewItemLabel.values[0] for i in df_train['electropodemniki'].dropna().unique()}
mask['nan'] = 'Нет данных'

In [ ]:
mask

In [ ]:
df_train['electropodemniki'] = df_train['electropodemniki'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['electropodemniki', 'price']], x="electropodemniki", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 0.5e7))
plt.show()

#### Summary

Почти половина значений - пропуски, при этом видно что распределения у категорий "передние и задние" и "только передние" заметно отличаются.
Для разметки отсутствующих данных, возможно, поможет размечать модой по аналогичным моделям с аналогичной комплектацией

### Lights
####  fary

In [ ]:
df_train['fary'].describe()

In [ ]:
df_train['fary'].value_counts()

In [ ]:
mask = {str(i): options[options['id'] == i].viewItemLabel.values[0] for i in df_train['fary'].dropna().unique()}
mask['nan'] = 'Нет данных'
mask

In [ ]:
df_train['fary'] = df_train['fary'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['fary', 'price']], x="fary", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 1e7))
plt.show()

#### Summary

Аналогично предыдущим колонкам, данная имеет 2/3 пропусков. Возможно поможет разметка модой по аналогичным моделям и комплектации.

### Climate control
#### upravlenie_klimatom

In [ ]:
df_train['upravlenie_klimatom'].describe()

In [ ]:
df_train['upravlenie_klimatom'].value_counts()

In [ ]:
mask = {str(i): options[options['id'] == i].viewItemLabel.values[0] for i in df_train['upravlenie_klimatom'].dropna().unique()}
mask['nan'] = 'Нет данных'
mask

In [ ]:
df_train['upravlenie_klimatom'] = df_train['upravlenie_klimatom'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['upravlenie_klimatom', 'price']], x="upravlenie_klimatom", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 0.5e7))
plt.show()

### Summary

Аналогично предыдущим колонкам, данная имеет 2/3 пропусков. Возможно поможет разметка модой по аналогичным моделям и комплектации, а также парсинг description.

### Power steering
#### usilitel_rul 

In [ ]:
df_train['usilitel_rul'].describe()

In [ ]:
df_train['usilitel_rul'].value_counts()

In [ ]:
mask = {str(i): options[options['id'] == i].viewItemLabel.values[0] for i in df_train['usilitel_rul'].dropna().unique()}
mask['nan'] = 'Нет данных'
mask

In [ ]:
df_train['usilitel_rul'] = df_train['usilitel_rul'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['usilitel_rul', 'price']], x="usilitel_rul", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 0.5e7))
plt.show()

#### Summary

Аналогично предыдущим колонкам, данная имеет половину пропусков. Возможно поможет разметка модой по аналогичным моделям и комплектации.
Дополнительно можно попробовать парсить description, модель и equipment по запросам EHPS, HPS, EPS.

### Audiosystem mult
#### audiosistema_mult

In [ ]:
df_train['audiosistema_mult'].describe()

In [ ]:
df_train['audiosistema_mult'].value_counts()

In [ ]:
mask = {str(i): options[options['id'] == float(i.strip('[]'))].viewItemLabel.values[0] for i in df_train['audiosistema_mult'].dropna().unique()}
mask['None'] = 'Нет данных'
mask

In [ ]:
df_train['audiosistema_mult'] = df_train['audiosistema_mult'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['audiosistema_mult', 'price']], x="audiosistema_mult", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 0.5e7))
plt.show()

In [ ]:
# Посмотрим пересечение - какое количество колонок аудиосистемы у автомобилей с сабвуфером
sns.histplot(data=df_train[df_train['audiosistema_mult'] == 'Сабвуфер']['audiosistema'])
plt.xticks(rotation=70)

In [ ]:
df_train[df_train['audiosistema_mult'] == 'Сабвуфер']['audiosistema'].value_counts()

#### Summary

Таким образом, данная колонка - по сути бинарный категориальный признак наличия сабвуфера в автомобильной аудиосистеме.
Пропущенные значения можно попытаться спарсить из description.

### Tares and wheels mult
#### shini_i_diski_mult

In [ ]:
df_train['shini_i_diski_mult'].describe()

In [ ]:
mask = {str(i): options[options['id'] == float(i.strip('[]'))].viewItemLabel.values[0] for i in df_train['shini_i_diski_mult'].dropna().unique()}
mask['None'] = 'Нет данных'
mask

In [ ]:
df_train['shini_i_diski_mult'] = df_train['shini_i_diski_mult'].apply(lambda x: mask[str(x)])

In [ ]:
fig = sns.catplot(data=df_train[['shini_i_diski_mult', 'price']], x="shini_i_diski_mult", y="price", kind="box", height=6, aspect=2)
plt.xticks(rotation=70)
fig.set(ylim=(0, 0.5e7))
plt.show()

#### Summary

Данная колонка так же является бинарным категориальным признаком наличия зимных шин в комплекте.
Пропущенные значения можно попробывать парсить из description.